# Méthodes

Chargement des bibliothèques nécéssaires

In [1]:
using DataFrames
using StatsBase
using DelimitedFiles, CSV
using MLJ
using Combinatorics, JuMP, GLPK
using BenchmarkTools

L'exemple illustratif

In [2]:
data = DataFrame(Taille=[2, 3, 1, 2, 3, 1, 1, 2, 3, 3],
Qualite=[3, 3, 3, 2, 1, 2, 1, 1, 2, 2],
Prix=[45, 45, 45, 55, 35, 30, 35, 30, 30, 55],
Choix=[3, 3, 2, 1, 2, 2, 1, 1, 3, 2])

attributes = ["Taille", "Qualite", "Prix"]
target = "Choix";

## DRSA

In [3]:
K = 3
PT = [2 3 2; 3 3 2; 1 3 2; 2 2 1; 3 1 3;
      1 2 4; 1 1 3; 2 1 4; 3 2 4; 3 2 1]

C = [3,3,2,1,2,2,1,1,3,2]
n, m = size(PT)

P = collect(powerset(collect(1:m),1))
sP = length.(P)
np = length(P)

D = permutedims(permutedims(repeat(PT,1,1,n),(3,2,1)) .>= PT, (3,1,2)) # dominance i x j for any criterion k

DP = [all(D[:,:,p], dims = 3) for p in P]

LAge = cat(
[
    reshape(
        permutedims(
            all(
                repeat(.!DP[i], 1, 1, K) .|| (repeat(DP[i] .* C, 1, 1, K) .>= reshape(collect(1:K),1,1,K))
                , dims = 1)
            , (3,2,1))
        , K, n)
    for i=1:np
]..., dims = 3) # lower approximation for >=

UAge = cat(
[
    reshape(
        permutedims(
            any(
                repeat(DP[i] .* C', 1, 1, K) .>= reshape(collect(1:K),1,1,K)
                , dims = 2)
            , (3,1,2))
        , K, n)
    for i=1:np
]..., dims = 3) # upper approximation for >=

BRge = UAge .&& .!LAge # boundary for >=

LAle = cat(
[
    reshape(
        permutedims(
            all(
                repeat(DP[i] .* C', 1, 1, K) .<= reshape(collect(1:K),1,1,K)
                , dims = 2)
            , (3,1,2))
        , K, n)
    for i=1:np
]..., dims = 3) # lower approximation for <=

UAle = cat(
[
    reshape(
        permutedims(
            any(
                repeat(DP[i], 1, 1, K) .&& (repeat(DP[i] .* C, 1, 1, K) .<= reshape(collect(1:K),1,1,K))
                , dims = 1)
            , (3,2,1))
        , K, n)
    for i=1:np
]..., dims = 3) # upper approximation for <=

BRle = UAle .&& .!LAle # boundary for <=

indices = []
cover = falses(n,0)
cost = []
for k = 1:K
    if k == 1
        X = vec(any(LAle[k,:,:], dims = 1))
        for i=1:np
            if X[i]
                Y = vec(all(LAle[k,:,:] .== LAle[k,:,i], dims = 1))
                if minimum(sP[Y]) == sP[i]
                    global cover = hcat(cover, LAle[k,:,i:i])
                    push!(indices, (k,[],P[i],true))
                    push!(cost, 1)
                end
            end
        end
    elseif k == K
        X = vec(any(LAge[k,:,:], dims = 1))
        for i=1:np
            if X[i]
                Y = vec(all(LAge[k,:,:] .== LAge[k,:,i], dims = 1))
                if minimum(sP[Y]) == sP[i]
                    global cover = hcat(cover, LAge[k,:,i:i])
                    push!(indices, (k,P[i],[],true))
                    push!(cost, 1)
                end
            end
        end
    else
        Z = repeat(LAge[k,:,:], 1, 1, np) .&& permutedims(repeat(LAle[k,:,:], 1, 1, np), (1, 3, 2))
        X = reshape(permutedims(any(Z, dims = 1), (2,3,1)), np, np)
        for i=1:np
            for j=1:np
                if X[i,j]
                    Yi = vec(all(Z[:,:,j] .== Z[:,i,j], dims = 1))
                    Yj = vec(all(Z[:,i,:] .== Z[:,i,j], dims = 1))
                    if minimum(sP[Yi]) == sP[i] && minimum(sP[Yj]) == sP[j]
                        global cover = hcat(cover, Z[:,i,j])
                        push!(indices, (k,P[i],P[j],true))
                        push!(cost, 1)
                    end
                end
            end
        end
    end
end

for k = 1:K-1
    for i=1:np
        if any(BRle[k,:,i])
            global cover = hcat(cover, BRle[k,:,i])
            push!(indices, (k,P[i],P[i],false))
            push!(cost, n+1)
        end
    end
end

v = length(indices)

model = JuMP.Model(GLPK.Optimizer)

@variable(model, x[1:v], Bin)

@constraint(model,[i=1:n], sum(x .* cover[i,:]) >= 1)

@objective(model, Min, sum(x[j] * cover[i,j] * cost[j] for i=1:n for j=1:v))

optimize!(model)

r = round.(Int,value.(x)) .== 1
cover[:,r] .* C
indices[r]

for j=1:v
    if r[j]
        X = cover[:,j]
        k, pge, ple, certain = indices[j]
        if certain
            if k == 1
                println("If ",join([join(["c$i"," <= ",maximum(PT[cover[:,j],i])]) for i in ple] ," and ")," then category $k")
            elseif k == K
                println("If ",join([join(["c$i"," >= ",minimum(PT[cover[:,j],i])]) for i in pge] ," and ")," then category $k")
            else
                println("If ",join(vcat([join(["c$i"," <= ",maximum(PT[cover[:,j],i])]) for i in ple],[join(["c$i"," >= ",minimum(PT[cover[:,j],i])]) for i in pge]) ," and ")," then category $k")
            end
        else
            #println("If ",join([join(["c$i"," <= ",maximum(PT[cover[:,j],i])]) for i in ple if all(PT[:,i])] ," and ")," then category ",join(["$l" for l=k:k+1]," or "))
            println("If ",join([join(["c$i"," <= ",maximum(PT[cover[:,j],i])]) for i in ple] ," and ")," then category ",join(["$l" for l=k:k+1]," or "))
        end
    end
end

If c1 <= 2 and c2 <= 2 and c3 <= 4 then category 1
If c1 <= 3 and c2 <= 3 and c3 <= 4 and c1 >= 1 and c2 >= 1 and c3 >= 1 then category 2
If c1 >= 2 and c2 >= 2 and c3 >= 2 then category 3


## Les arbres de décisions

La structure utilisée pour chaque noeud (sauf les noeuds feuilles) des arbres de décision 

In [3]:
struct Node
    feature::String # attribut
    children::Dict # noeud fils
end

### Les différentes fonctions utilisées pour les critères de sélection de l'attribut



$$$$
Entropie : $$ entropie (T)= \displaystyle\sum_{C \in D_{C_l}} - \mathbb{P}(T(C_l) = C) * log_2(\mathbb{P}(T(C_l) = C))$$

In [4]:
function log_2(x)
    if x == 0
        return x
    else
        return log2(x)
    end
end

# Fonction pour calculer l'entropie
function entropy(data, target)
    b = combine(groupby(data, target), nrow) # proportion des différentes classes dans data
    probabilities = b[:,2] ./ size(data)[1] # probabilité de chaque classe
    entropy = -sum(probabilities .* log_2.(probabilities))
    return entropy
end

entropy (generic function with 1 method)

$$$$Score d'entropie : $$E(At) = \displaystyle\sum _{v \in D_{At}} \frac{n_v}{n} entropie(T_v)$$

In [5]:
function score_e(data, feature, target)
    feature_values = unique(data[:,feature]) # les différentes valeurs de l'attribut
    weighted_entropy = 0.0
    for value in feature_values
        subset = filter(feature => ==(value), data) # sous-ensemble de data 
        subset_entropy = entropy(subset, target)
        weighted_entropy += (size(subset)[1] / size(data)[1]) * subset_entropy
    end
    return weighted_entropy
end

score_e (generic function with 1 method)

$$$$

Gain d'information : $$gain (At) = entropie(T) - E(At)$$

In [6]:
function information_gain(data, feature, target)
    total_entropy = entropy(data, target)
    score = score_e(data, feature, target)
    return total_entropy - score
end

information_gain (generic function with 1 method)

$$$$Indice de Gini : $$ gini (T) = 1 - \displaystyle\sum _{C \in D_{C_l}} \mathbb{P}(T(C_l) = C)^2$$ 

In [7]:
function gini_index(data, target)
    b = combine(groupby(data, target), nrow)
    probabilities = b[:,2] ./ size(data,1)
    gini = 1.0 - sum(probabilities .^2)
    return gini
end

gini_index (generic function with 1 method)

$$$$Score de Gini $$ S_ {gini} = \frac{n_L*gini(T_L) + n_R*gini(T_R)}{n}$$

In [8]:
function gini_score(data, target, feature, value)
    left_subset = filter(feature => <=(value), data)
    index_left = gini_index(left_subset, target)

    right_subset = filter(feature => >(value), data)
    index_right = gini_index(right_subset, target)

    gini = (index_left*size(left_subset,1) + index_right*size(right_subset,1))/size(data,1)
    return gini
end

gini_score (generic function with 1 method)

$$$$Ranking Impurity $$I_{rank}(T)  = \displaystyle\sum _{j=1}^k \displaystyle\sum _{i=1}^j (j-i) N_j (T) N_i (T)$$ 

In [9]:
function ranking_impurity(data, target)
    N = combine(groupby(sort(data,target), target), nrow)
    n = size(N,1)
    impurity = 0.0
    for j in 1:n
        for i in 1:j
            impurity += (j-i)*N[j,2]*N[i,2]
        end
    end
    return impurity
end

ranking_impurity (generic function with 1 method)

### ID3

In [10]:
function ID3(data, attributes, target)
    if size(unique(data[:,target]),1) == 1
        # Cas de base : si tous les exemples ont la même classe
        # retourner un nœud avec cette classe
        return data[1,target]
    end
    
    if length(attributes) == 0
        # Cas de base : si toutes les caractéristiques ont été utilisées
        # retourner un nœud avec la classe majoritaire
        return StatsBase.mode(data[:, target])
    end
    
    # Sélection de la meilleure caractéristique pour la division
    best_feature = ""
    best_score = Inf
    for feature in attributes
        score = score_e(data, feature, target)
        if best_score > score
            best_score = score
            best_feature = feature
        end
    end

    if isempty(best_feature)
        return StatsBase.mode(data[:,target])
    end

    # Création du nœud de décision avec la meilleure caractéristique
    tree = Node(best_feature, Dict())
    subset_attributes = filter(x -> x != best_feature, attributes)
    
    # Séparation des exemples en fonction des valeurs de la meilleure caractéristique
    feature_values = sort(unique(data[:,best_feature]))
    for value in feature_values
        sub = filter(best_feature => ==(value), data)
        subset = select(sub, Not(best_feature))
        # Construction récursive de l'arbre pour le sous-ensemble
        tree.children[value] = ID3(subset, subset_attributes, target)
    end    
    return tree
end

ID3 (generic function with 1 method)

In [11]:
tree1 = ID3(data, attributes, target)

Node("Taille", Dict{Any, Any}(2 => Node("Qualite", Dict{Any, Any}(2 => 1, 3 => 3, 1 => 1)), 3 => Node("Prix", Dict{Any, Any}(35 => 2, 55 => 2, 45 => 3, 30 => 3)), 1 => Node("Qualite", Dict{Any, Any}(2 => 2, 3 => 2, 1 => 1))))

### C4.5

Fonction pour trouver le seuil optimal d'un attribut continu

In [12]:
function Threshold(data, attr, order, target)
    A = sort(unique(data[:,attr]))
    n = length(A)
    S = zeros(n-1,1)
    if order == "c"
        for i in 1:n-1
            S[i] = (A[i] + A[i+1])/2
        end
    elseif order == "d"
        for i in n:-1:2
            S[n-i+1] = (A[i] + A[i-1])/2
        end
    end
    
    best_info_gain = 0.0
    best_threshold = -Inf
    total_entropy = entropy(data, target)
    attr_entropy = entropy(data, attr)
    for i in 1:n-1
        left_subset = filter(attr => <=(S[i]), data)
        right_subset = filter(attr => >(S[i]), data)
        left_entropy = entropy(left_subset, target)
        right_entropy = entropy(right_subset, target)
        
        weighted_entropy = (size(left_subset,1) / size(data,1) * left_entropy) + (size(right_subset,1) / size(data,1) * right_entropy)
        
        # Calcule du gain d'information
        info_gain = total_entropy - weighted_entropy

        # Calcule du gain ratio
        gain_ratio = info_gain / attr_entropy
        
        if gain_ratio > best_info_gain
            best_info_gain = gain_ratio
            best_threshold = S[i]
        end
    end
    return best_threshold
end

Threshold (generic function with 1 method)

In [13]:
function C4_5(data, target, attributes, max_depth=Inf, min_samples=1, depth=0)
    
    # Vérifier les conditions d'arrêt (profondeur maximale ou nombre minimal d'échantillons)
    if depth >= max_depth || nrow(data) <= min_samples
        # Créer un nœud feuille avec la classe majoritaire dans l'ensemble de données
        return StatsBase.mode(data[:, target])
    end

    if size(unique(data[:,target]),1) == 1
        # Cas de base : si tous les exemples ont la même classe
        # retourner un nœud avec cette classe
        return data[1,target]
    end
    
    if length(attributes) == 0
        # Cas de base : si toutes les caractéristiques ont été utilisées
        # retourner un nœud avec la classe majoritaire
        return StatsBase.mode(data[:, target])
    end

    # Sélection de la meilleure caractéristique pour la division
    best_info_gain = 0.0
    best_attribute = ""
    
    for attribute in attributes
        # Calcule du gain d'information
        info_gain = information_gain(data, attribute, target)

        # Calcule de l'entropie de l'attribut
        attr_entropy = entropy(data, attribute)

        # Calcule du gain ratio
        gain_ratio = info_gain / attr_entropy

        if gain_ratio > best_info_gain
            best_info_gain = gain_ratio
            best_attribute = attribute
        end
    end
    
    if isempty(best_attribute)
        return StatsBase.mode(data[:,target])
    end

    # Création du nœud de décision avec la meilleure caractéristique
    tree = Node(best_attribute, Dict())
    depth += 1
    subset_attributes = filter(x -> x != best_attribute, attributes)

    # Séparation des exemples en fonction des valeurs de la meilleure caractéristique
    feature_values = sort(unique(data[:,best_attribute]))
    for value in feature_values
        sub = filter(best_attribute => ==(value), data)
        subset = select(sub, Not(best_attribute))
        # Construction récursive de l'arbre pour le sous-ensemble
        tree.children[value] = C4_5(subset, target, subset_attributes, max_depth, min_samples, depth)
    end
    
    return tree
end

C4_5 (generic function with 4 methods)

$$$$Version pour des données continues

In [14]:
# Fonction pour la recherche du meilleur seuil
function Threshold1(data, attr, target)
    A = sort(unique(data[:,attr]))
    n = length(A)
    S = zeros(n-1,1)
    for i in 1:n-1
        # Calcul de chaque seuil
        S[i] = (A[i] + A[i+1])/2 
    end
    
    best_info_gain = 0.0
    best_threshold = -Inf
    total_entropy = entropy(data, target)
    attr_entropy = entropy(data, attr)
    for i in 1:n-1
        # Scission des données en deux 
        left_subset = filter(attr => <=(S[i]), data)
        right_subset = filter(attr => >(S[i]), data)
        # Entropie de chaque sous-ensemble
        left_entropy = entropy(left_subset, target)
        right_entropy = entropy(right_subset, target)
        
        weighted_entropy = (size(left_subset,1) / size(data,1) * left_entropy) + (size(right_subset,1) / size(data,1) * right_entropy)
        
        # Calcule du gain d'information
        info_gain = total_entropy - weighted_entropy

        # Calcule du gain ratio
        gain_ratio = info_gain / attr_entropy
        
        if gain_ratio > best_info_gain
            best_info_gain = gain_ratio
            best_threshold = S[i]
        end
    end
    return best_threshold, best_info_gain
end

# C4.5 pour des données continues
function C4_5_con(data, target, attributes, max_depth=Inf, min_samples=1, depth=0)
    
    # Vérifier les conditions d'arrêt (profondeur maximale ou nombre minimal d'échantillons)
    if depth >= max_depth || nrow(data) <= min_samples
        # Créer un nœud feuille avec la classe majoritaire dans l'ensemble de données
        return StatsBase.mode(data[:, target])
    end

    if length(unique(data[:,target])) == 1
        # Cas de base : si tous les exemples ont la même classe
        # retourner un nœud avec cette classe
        return data[1,target]
    end

    # Sélection de la meilleure caractéristique pour la division
    best_info_gain = 0.0
    best_attribute = ""
    best_threshold = -Inf
    for attribute in attributes
        (threshold, gain_ratio) = Threshold1(data, attribute, target)
        if gain_ratio > best_info_gain
            best_info_gain = gain_ratio
            best_attribute = attribute
            best_threshold = threshold
        end
    end
    
    if isempty(best_attribute)
        return StatsBase.mode(data[:,target])
    end

    # Partitionner les exemples en fonction du seuil
    left_subset = filter(best_attribute => <=(best_threshold), data)
    right_subset = filter(best_attribute => >(best_threshold), data)
    value = sort(unique(right_subset[:,best_attribute]))[1]
    
    # Création du nœud de décision avec la meilleure caractéristique
    tree = Node(best_attribute, Dict())
    depth += 1

    # Séparation des exemples en fonction des valeurs du meilleur seuil
    tree.children[best_threshold] = C4_5_con(left_subset, target, attributes, max_depth, min_samples, depth)
    tree.children[value] = C4_5_con(right_subset, target, attributes, max_depth, min_samples, depth)
    
    return tree
end

C4_5_con (generic function with 4 methods)

In [15]:
attr = "Prix"
order = "d"

T = Threshold(data, attr, order, target)
DATA = copy(data)
for i in 1:size(data,1)
    if DATA[i,attr] <= T
        DATA[i,attr] = 2
    else
        DATA[i,attr] = 1
    end
end

tree2 = C4_5(DATA, target, attributes)

Node("Taille", Dict{Any, Any}(2 => Node("Qualite", Dict{Any, Any}(2 => 1, 3 => 3, 1 => 1)), 3 => Node("Prix", Dict{Any, Any}(2 => Node("Qualite", Dict{Any, Any}(2 => 3, 3 => 3, 1 => 2)), 1 => 2)), 1 => Node("Qualite", Dict{Any, Any}(2 => 2, 3 => 2, 1 => 1))))

### CART

In [16]:
function cart(data, target, attributes, max_depth=Inf, min_samples=1, depth=0)
    # Vérifier les conditions d'arrêt (profondeur maximale ou nombre minimal d'échantillons)
    if depth >= max_depth || nrow(data) <= min_samples
        # Créer un nœud feuille avec la classe majoritaire dans l'ensemble de données
        return StatsBase.mode(data[:, target])
    end
    
    # Vérifier si tous les exemples appartiennent à la même classe
    classes = unique(data[:,target])
    if size(classes,1) == 1
        return classes[1]
    end
    
    # Sélectionner l'attribut et la valeur de découpe qui maximisent le gain de Gini
    best_attribute, best_value, best_gini = "", 0, Inf
    for attribute in attributes
        values = sort(unique(data[:,attribute]))
        for value in values
            gini = gini_score(data, target, attribute, value)
            if best_gini > gini
                best_attribute = attribute
                best_value = value
                best_gini = gini
            end
        end
    end

    # Vérifier si les attributs sont vides
    if isempty(best_attribute)
        return StatsBase.mode(data[:,target])
    end
    
    # Créer le noeud de décision
    node = Node(string(best_attribute), Dict())
    depth += 1
    
    # Partitionner les exemples en fonction de la valeur de découpe sélectionnée
    left_subset = filter(best_attribute => <=(best_value), data)
    right_subset = filter(best_attribute => >(best_value), data)
    
    # Construction récursive des sous-arbres gauche et droit
    if isempty(left_subset) || isempty(right_subset)
        return StatsBase.mode(data[:,target])
    else
        # Créer le noeud de décision
        node = Node(string(best_attribute), Dict())
        value = sort(unique(right_subset[:,best_attribute]))[1]
        depth += 1
        node.children[best_value] = cart(left_subset, target, attributes, max_depth, min_samples, depth)
        node.children[value] = cart(right_subset, target, attributes, max_depth, min_samples, depth)
    end
    
    return node
end


cart (generic function with 4 methods)

In [17]:
tree3 = cart(data, target, attributes)

Node("Qualite", Dict{Any, Any}(2 => Node("Taille", Dict{Any, Any}(2 => Node("Prix", Dict{Any, Any}(55 => Node("Taille", Dict{Any, Any}(2 => 1, 3 => 2)), 45 => 3)), 1 => 2)), 1 => Node("Taille", Dict{Any, Any}(2 => 1, 3 => 2))))

### Ranking Tree 

In [18]:
function RankTree(data, target, attributes, max_depth=Inf, min_samples=1, depth=0)
    # Vérifier les conditions d'arrêt (profondeur maximale ou nombre minimal d'échantillons)
    if depth >= max_depth || nrow(data) <= min_samples
        # Créer un nœud feuille avec la classe majoritaire dans l'ensemble de données
        return StatsBase.mode(data[:, target])
    end
    
    # Vérifier si tous les exemples appartiennent à la même classe
    classes = unique(data[:,target])
    if size(classes,1) == 1
        return classes[1]
    end
    
    # Sélectionner l'attribut et la valeur de découpe qui maximisent le gain de Gini
    best_attribute = ""
    best_value = 0
    best_decrease = Inf
    for attribute in attributes
        values = sort(unique(data[:,attribute]))
        for value in values[1:end-1]

            left_child = filter(attribute => <=(value), data)
            left_I = ranking_impurity(left_child, target)

            right_child = filter(attribute => >(value), data)
            right_I = ranking_impurity(right_child, target)

            score_I = left_I + right_I

            if score_I < best_decrease
                best_decrease = score_I
                best_attribute = attribute
                best_value = value   
            end
        end
    end
    
    if isempty(best_attribute)
        return StatsBase.mode(data[:,target])
    end
    
    # Partitionner les exemples en fonction de la valeur de découpe sélectionnée
    left_subset = filter(best_attribute => <=(best_value), data)
    right_subset = filter(best_attribute => >(best_value), data)
    
    # Construction récursive des sous-arbres gauche et droit
    if isempty(left_subset) || isempty(right_subset)
        return StatsBase.mode(data[:,target])
    else
        # Créer le noeud de décision
        node = Node(string(best_attribute), Dict())
        value = sort(unique(right_subset[:,best_attribute]))[1]
        depth += 1
        node.children[best_value] = RankTree(left_subset, target, attributes, max_depth, min_samples, depth)
        node.children[value] = RankTree(right_subset, target, attributes, max_depth, min_samples, depth)
    end
   
    return node
end


RankTree (generic function with 4 methods)

In [19]:
tree4 = RankTree(data, target, attributes)

Node("Taille", Dict{Any, Any}(2 => Node("Taille", Dict{Any, Any}(2 => Node("Qualite", Dict{Any, Any}(2 => 1, 3 => 3)), 1 => Node("Qualite", Dict{Any, Any}(2 => 2, 1 => 1)))), 3 => Node("Qualite", Dict{Any, Any}(2 => Node("Prix", Dict{Any, Any}(55 => 2, 45 => 3)), 1 => 2))))

### Fonctions de prédiction

In [20]:
function predict_tree(tree, example)
    if typeof(tree)==Int
        return tree
    else
        value = example[tree.feature]
        k = []
        for i in keys(tree.children)
            push!(k,i)
        end
        i = length(k)
        k = sort(k)
        while value < k[i] && i!=1
            i-=1
        end
        subtree = tree.children[k[i]]
        return predict_tree(subtree, example)
    end
end

# Arbre de décision binaire
function predict_bi(tree, example)
    if typeof(tree)==Int
        return tree
    else
        k = []
        for i in keys(tree.children)
            push!(k,i)
        end
        k = sort(k)
        if example[tree.feature] <= k[1]
            return predict_bi(tree.children[k[1]], example)
        else
            return predict_bi(tree.children[k[2]], example)
        end
    end
end

predict_bi (generic function with 1 method)

### Forêt aléatoire

In [21]:
function bootstrap_sample(data, n_samples)
    n = size(data, 1)
    indices = rand(1:n, n_samples)
    return data[indices, :]
end

bootstrap_sample (generic function with 1 method)

In [22]:
function Forest_RI(data, ntree, n_samples, attributes, target)
    trees = DataFrame()
    trees[!,"tree"] = Vector{Any}(missing, ntree)
    n = size(data, 1)
    for i in 1:ntree
        data_2 = bootstrap_sample(data, n_samples)
        trees[i,"tree"] = cart(data_2, target, attributes)
    end
    return trees
end

Forest_RI (generic function with 1 method)

In [24]:
forest1 = Forest_RI(data, 10, 5, attributes, target)

Row,tree
,Any
1,2
2,"Node(""Prix"", Dict{Any, Any}(35=>Node(""Taille"", Dict{Any, Any}(3=>3, 1=>1)), 45=>2))"
3,"Node(""Taille"", Dict{Any, Any}(2=>Node(""Qualite"", Dict{Any, Any}(2=>1, 3=>3)), 3=>Node(""Prix"", Dict{Any, Any}(55=>2, 30=>3))))"
4,"Node(""Taille"", Dict{Any, Any}(2=>Node(""Qualite"", Dict{Any, Any}(2=>1, 3=>3)), 3=>2))"
5,"Node(""Taille"", Dict{Any, Any}(2=>3, 1=>2))"
6,"Node(""Qualite"", Dict{Any, Any}(2=>Node(""Taille"", Dict{Any, Any}(2=>Node(""Prix"", Dict{Any, Any}(55=>2, 45=>3)), 1=>2)), 1=>1))"
7,"Node(""Taille"", Dict{Any, Any}(3=>Node(""Qualite"", Dict{Any, Any}(2=>3, 1=>2)), 1=>2))"
8,"Node(""Qualite"", Dict{Any, Any}(2=>Node(""Taille"", Dict{Any, Any}(2=>Node(""Taille"", Dict{Any, Any}(2=>1, 1=>2)), 3=>2)), 3=>3))"
9,"Node(""Qualite"", Dict{Any, Any}(3=>3, 1=>1))"


$$$$Fonction de prédiction à partir d'une forêt aléatoire

In [25]:
function predict_rf(trees, example)
    ntree = size(trees,1)
    pred = zeros(Int, ntree)
    
    for i in 1:ntree
        pred[i] = predict_bi(trees[i,1], example)
    end
    
    return StatsBase.mode(pred)
end

predict_rf (generic function with 1 method)

### Monotonie

$$$$L’indice de non-monotonie : $$\text{INM}(arbre) = \frac{\sum _{i=1}^N \sum _{j=1}^N m_{i,j}}{N^2 - N}$$

Où $m_{i,j}$ est égale à 1 si la paire de branches $(b_i,\, b_j)$ est non-monotone et $N$ est le nombre total de branches.

In [26]:
function NMI(tab)
    (n,m) = size(tab)
    W = 0
    for i in 1:n-1
        for j in i+1:n
            inf = 0
            sup = 0
            eq = 0
            for k in 1:m-1
                if (!ismissing(tab[i,k]) && !ismissing(tab[j,k]))
                    if tab[i,k] < tab[j,k]
                        inf += 1
                    elseif tab[i,k] > tab[j,k]
                        sup += 1
                    else
                        eq += 1
                    end
                end
            end
            if (sup==0 && tab[i,m] > tab[j,m])
                W += 1
            elseif (inf == 0 && tab[i,m] < tab[j,m])
                W += 1
            elseif (eq==m-1 && tab[i,m] != tab[j,m])
                W += 1
            end
        end
    end
    return W*2 / (n^2-n)
end

NMI (generic function with 1 method)

$$$$

Le score d'ambiguïté de l'ordre  : 
\begin{cases}
0 &\quad \text{si } INM = 0 \\
- (\text{log}_2 INM)^{-1} &\quad sinon
  \end{cases}

In [27]:
function score_a(INM)
    if INM == 0
        return 0
    else
        return -log2(INM)^(-1)
    end
end    

score_a (generic function with 1 method)

$$$$Fonction pour pouvoir aplliquer la fonction de l'indice de non-monotonie à un arbre de décision

In [28]:
# Fonction récursive pour extraire les différentes branches de l'arbre
function branchs(node, path, paths)
    if typeof(node)==Int
        current_path = [path..., node]
        push!(paths, [current_path])
    else
        current_path = [path..., node.feature]
        for (value, child_node) in pairs(node.children)
            branchs(child_node, [current_path..., value], paths)
        end
    end
end

# Fonction pour extraire les éléments des branches dans un tableau 
function data_tree(paths, attributes, target)

    n = size(paths,1)
    tab = DataFrame()
    for value in attributes
        tab[!,value] = Vector{Any}(missing, n)
    end
    tab[!, target] = Vector{Any}(missing, n)
    
    for i in 1:n
        m = length(paths[i,1])
        for j in 1:Int((m-1)/2)
            tab[i, paths[i,1][2*j-1]] = paths[i,1][2*j]
        end
        tab[i,target] = paths[i,1][m]
    end

    return tab
end

data_tree (generic function with 1 method)

### CART monotone

In [29]:
function mon_cart(data, target, attributes, path, branch, R=1, max_depth=Inf, min_samples=1, depth=0)
    # Vérifier les conditions d'arrêt (profondeur maximale ou nombre minimal d'échantillons)
    if depth >= max_depth || nrow(data) <= min_samples
        majority_class = StatsBase.mode(data[:,target])
        current_path = [path..., majority_class]
        push!(branch, [current_path])
        # Créer un nœud feuille avec la classe majoritaire dans l'ensemble de données
        return majority_class
    end
    # Vérifier si tous les exemples appartiennent à la même classe
    classes = unique(data[:,target])
    if length(classes) == 1
        current_path = [path..., classes[1]]
        push!(branch, [current_path])
        return classes[1]
    end
       
    if !isempty(branch)
        tab = data_tree(branch, attributes, target)
        push!(tab, Vector{Any}(missing, size(tab,2)))
    else
        tab = DataFrame()
        for value in attributes
            tab[!,value] = Vector{Any}(missing, 1)
        end
        tab[!, target] = Vector{Any}(missing, 1)
    end
    
    # Sélectionner l'attribut et la valeur de découpe qui maximisent le score d'ambiguïté totale
    best_attribute = ""
    best_value = 0
    best_score = Inf
    for attribute in attributes
        T = copy(tab)
        push!(T, Vector{Any}(missing, size(T,2)))
        if !isempty(path)
            m = length(path)
            for j in 1:Int((m)/2)
                T[end-1:end, path[2*j-1]] .= path[2*j]
            end
        end
        values = sort(unique(data[:,attribute]))
        n = size(values,1)
        for i in 1:n-1
            T[end-1,attribute] = values[i]
            left_child = filter(attribute => <=(values[i]), data)
            T[end-1,target] = StatsBase.mode(left_child[:,target])

            right_child = filter(attribute => >(values[i]), data)
            T[end,attribute] = values[i+1]
            T[end,target] = StatsBase.mode(right_child[:,target])
            
            index_left = gini_index(left_child, target)
            index_right = gini_index(right_child, target)

            gini = (index_left*size(left_child,1) + index_right*size(right_child,1))/size(data,1)
            INM = NMI(T)
            score = gini + R*score_a(INM)
            if best_score > score
                best_attribute = attribute
                best_value = values[i]
                best_score = score
            end
        end
    end
    
    if isempty(best_attribute)
        majority_class = StatsBase.mode(data[:,target])
        current_path = [path..., majority_class]
        push!(branch, [current_path])
        return majority_class
    end

    # Partitionner les exemples en fonction de la valeur de découpe sélectionnée
    left_subset = filter(best_attribute => <=(best_value), data)
    right_subset = filter(best_attribute => >(best_value), data)
    
    # Construction récursive des sous-arbres gauche et droit
    if isempty(left_subset) || isempty(right_subset)
        return StatsBase.mode(data[:,target])
    else
        # Créer le noeud de décision
        node = Node(string(best_attribute), Dict())
        value = sort(unique(right_subset[:,best_attribute]))[1]
        depth += 1
        current_path = [path..., best_attribute]
        node.children[best_value] = mon_cart(left_subset, target, attributes, [current_path..., best_value], branch, R, max_depth, min_samples, depth)
        node.children[value] = mon_cart(right_subset, target, attributes, [current_path..., value], branch, R, max_depth, min_samples, depth)
    end
    
    return node
end

mon_cart (generic function with 5 methods)

In [30]:
data_or = DataFrame(Taille=[2, 3, 1, 2, 3, 1, 1, 2, 3, 3],
Qualite=[3, 3, 3, 2, 1, 2, 1, 1, 2, 2],
Prix=[2, 2, 2, 1, 3, 4, 3, 4, 4, 1],
Choix=[3, 3, 2, 1, 2, 2, 1, 1, 3, 2]);

In [31]:
branch = DataFrame(col=Vector())
tree5 = mon_cart(data_or, target, attributes, [], branch)

Node("Prix", Dict{Any, Any}(2 => Node("Qualite", Dict{Any, Any}(2 => Node("Taille", Dict{Any, Any}(2 => 1, 3 => 2)), 3 => Node("Taille", Dict{Any, Any}(2 => 3, 1 => 2)))), 3 => Node("Qualite", Dict{Any, Any}(2 => Node("Taille", Dict{Any, Any}(3 => 3, 1 => 2)), 1 => Node("Taille", Dict{Any, Any}(2 => 1, 3 => 2))))))

### ID3 monotone

In [32]:
function MON_ID3(data, target, attributes, path, branch, R=1, max_depth=Inf, min_samples=1, depth=0)
    # Vérifier les conditions d'arrêt (profondeur maximale ou nombre minimal d'échantillons)
    if depth >= max_depth || nrow(data) <= min_samples
        current_path = [path..., StatsBase.mode(data[:, target])]
        push!(branch, [current_path])
        # Créer un nœud feuille avec la classe majoritaire dans l'ensemble de données
        return StatsBase.mode(data[:, target])
    end

    if  size(unique(data[:,target]),1) == 1
        # Cas de base : si tous les exemples ont la même classe
        current_path = [path..., data[1,target]]
        push!(branch, [current_path])
        # retourner un nœud avec cette classe
        return data[1,target]
    end
    
    if length(attributes) == 0
        # Cas de base : si toutes les caractéristiques ont été utilisées
        current_path = [path..., StatsBase.mode(data[:, target])]
        push!(branch, [current_path])
        # retourner un nœud avec la classe majoritaire
        return StatsBase.mode(data[:, target])
    end
    
    if !isempty(branch)
        tab = data_tree(branch, attributes, target)
        push!(tab, Vector{Any}(missing, size(tab,2)))
    else
        tab = DataFrame()
        for value in attributes
            tab[!,value] = Vector{Any}(missing, 1)
        end
        tab[!, target] = Vector{Any}(missing, 1)
    end
    
    # Sélection de la meilleure caractéristique pour la division
    best_feature = ""
    best_score = Inf
    for feature in attributes
        values = unique(data[:,feature])
        n = length(values)
        if n != 1
            T = copy(tab)
            for i in 1:n-1
                push!(T, zeros(size(T,2)))
            end
            if !isempty(path)
                m = length(path)
                for j in 1:Int((m)/2)
                    T[end-n+1:end, path[2*j-1]] .= path[2*j]
                end
            end
            score = 0.0
            for i in 1:n
                T[end-i+1,feature] = values[i]
                subset = filter(feature => ==(values[i]), data)
                T[end-i+1,target] = StatsBase.mode(subset[:,feature])
                score += (size(subset,1) / size(data,1)) * entropy(subset, target)
            end
        
            INM = NMI(T)
            score += R*score_a(INM)
            if best_score > score
                best_score = score
                best_feature = feature
            end
        end
    end

    if isempty(best_feature)
        majority_class = StatsBase.mode(data[:,target])
        current_path = [path..., majority_class]
        push!(branch, [current_path])
        return majority_class
    end

    # Création du nœud de décision avec la meilleure caractéristique
    tree = Node(best_feature, Dict())
    depth += 1
    current_path = [path..., best_feature]
    
    # Séparation des exemples en fonction des valeurs de la meilleure caractéristique
    feature_values = sort(unique(data[:,best_feature]))
    for value in feature_values
        subset = filter(best_feature => ==(value), data)
        # Construction récursive de l'arbre pour le sous-ensemble
        tree.children[value] = MON_ID3(subset, target, attributes, [current_path..., value], branch, R, max_depth, min_samples, depth)
    end    
    return tree
end

MON_ID3 (generic function with 5 methods)

In [33]:
branch = DataFrame(col=Vector())
tree6 = MON_ID3(data_or, target, attributes, [], branch, 2)

Node("Taille", Dict{Any, Any}(2 => Node("Qualite", Dict{Any, Any}(2 => 1, 3 => 3, 1 => 1)), 3 => Node("Prix", Dict{Any, Any}(4 => 3, 2 => 3, 3 => 2, 1 => 2)), 1 => Node("Qualite", Dict{Any, Any}(2 => 2, 3 => 2, 1 => 1))))

### C4.5 monotone

In [34]:
function MON_C4_5(data, target, attributes, path, branch, R=1, max_depth=Inf, min_samples=1, depth=0)
    
    # Vérifier les conditions d'arrêt (profondeur maximale ou nombre minimal d'échantillons)
    if depth >= max_depth || nrow(data) <= min_samples
        current_path = [path..., StatsBase.mode(data[:, target])]
        push!(branch, [current_path])
        # Créer un nœud feuille avec la classe majoritaire dans l'ensemble de données
        return StatsBase.mode(data[:, target])
    end

    if size(unique(data[:,target]),1) == 1
        # Cas de base : si tous les exemples ont la même classe
        current_path = [path..., data[1,target]]
        push!(branch, [current_path])
        # retourner un nœud avec cette classe
        return data[1,target]
    end
    
    if !isempty(branch)
        tab = data_tree(branch, attributes, target)
        push!(tab, Vector{Any}(missing, size(tab,2)))
    else
        tab = DataFrame()
        for value in attributes
            tab[!,value] = Vector{Any}(missing, 1)
        end
        tab[!, target] = Vector{Any}(missing, 1)
    end

    # Sélection de la meilleure caractéristique pour la division
    best_info_gain = 0.0
    best_attribute = ""
    total_entropy = entropy(data, target)
    for attribute in attributes
        values = unique(data[:,attribute])
        n = size(values,1)
        if n != 1
            T = copy(tab)
            for i in 1:n-1
                push!(T, Vector{Any}(missing, size(T,2)))
            end
            if !isempty(path)
                m = length(path)
                for j in 1:Int((m)/2)
                    T[end-n+1:end, path[2*j-1]] .= path[2*j]
                end
            end
            score = 0.0
            for i in 1:n
                T[end-i+1,attribute] = values[i]
                subset = filter(attribute => ==(values[i]), data)
                T[end-i+1,target] = StatsBase.mode(subset[:,attribute])
                score += (size(subset,1) / size(data,1)) * entropy(subset, target)
            end
        
            INM = NMI(T)
            # Calcule du gain d'information
            info_gain = total_entropy - (score + R*score_a(INM))
        
            # Calcule de l'entropie de l'attribut
            attr_entropy = entropy(data, attribute)

            # Calcule du gain ratio
            gain_ratio = info_gain / attr_entropy
        
            if gain_ratio > best_info_gain
                best_info_gain = gain_ratio
                best_attribute = attribute
            end
        end
    end

    if isempty(best_attribute)
        majority_class = StatsBase.mode(data[:,target])
        current_path = [path..., majority_class]
        push!(branch, [current_path])
        return majority_class
    end

    # Création du nœud de décision avec la meilleure caractéristique
    tree = Node(best_attribute, Dict())
    depth += 1
    current_path = [path..., best_attribute]

    # Séparation des exemples en fonction des valeurs de la meilleure caractéristique
    feature_values = sort(unique(data[:,best_attribute]))
    for value in feature_values
        subset = filter(best_attribute => ==(value), data)
        # Construction récursive de l'arbre pour le sous-ensemble
        tree.children[value] = MON_C4_5(subset, target, attributes, [current_path..., value], branch, R, max_depth, min_samples, depth)
    end
    
    return tree
end

MON_C4_5 (generic function with 5 methods)

$$$$Version pour des données continues croissantes

In [35]:
function Threshold2(data, attr, target, T)
    A = sort(unique(data[:,attr]))
    n = length(A)
    S = zeros(n,1) 
    for i in 1:n-1
        S[i] = (A[i] + A[i+1])/2 # les différents seuils
    end
    S[n] = A[n]
    best_info_gain = Inf
    best_threshold = -Inf
    total_entropy = entropy(data, target)
    attr_entropy = entropy(data, attr)
    for i in 1:n-1
        # Les branches en construction
        T[end-1,attr] = S[i]
        T[end,attr] = S[i+1]
        # Les sous-ensembles
        left_subset = filter(attr => <=(S[i]), data)
        right_subset = filter(attr => >(S[i]), data)
        # Le noeud feuille des branches en construction
        T[end-1,target] = StatsBase.mode(left_subset[:,target])
        T[end,target] = StatsBase.mode(right_subset[:,target])
        
        left_entropy = entropy(left_subset, target)
        right_entropy = entropy(right_subset, target)
        # Calcul de l'indice de non-monotonie
        INM = NMI(T)
        
        weighted_entropy = (size(left_subset,1) / size(data,1) * left_entropy) + (size(right_subset,1) / size(data,1) * right_entropy)
        # Calcule du gain d'information
        info_gain = weighted_entropy + R*score_a(INM)
        # Calcule du gain ratio
        gain_ratio = info_gain / attr_entropy
        
        if best_info_gain > gain_ratio
            best_info_gain = gain_ratio
            best_threshold = S[i]
        end
    end
    return best_threshold, best_info_gain
end

# Fonction pour construire l'arbre de décision
function MON_C4_5_con(data, target, attributes, path, branch, R=1, max_depth=Inf, min_samples=1, depth=0)
    # Vérifier les conditions d'arrêt (profondeur maximale ou nombre minimal d'échantillons)
    if depth >= max_depth || nrow(data) <= min_samples
        current_path = [path..., StatsBase.mode(data[:, target])]
        push!(branch, [current_path])
        # Créer un nœud feuille avec la classe majoritaire dans l'ensemble de données
        return StatsBase.mode(data[:, target])
    end

    if size(unique(data[:,target]),1) == 1
        # Cas de base : si tous les exemples ont la même classe
        current_path = [path..., data[1,target]]
        push!(branch, [current_path])
        # retourner un nœud avec cette classe
        return data[1,target]
    end
    
    if !isempty(branch)
        # Transformation de l'arbre déjà construit en tableau de branches
        tab = data_tree(branch, attributes, target)
        push!(tab, Vector{Any}(missing, size(tab,2)))
    else
        # Création d'un tableau de branches
        tab = DataFrame()
        for value in attributes
            tab[!,value] = Vector{Any}(missing,1)
        end
        tab[!, target] =  Vector{Any}(missing,1)
    end

    # Sélection de la meilleure caractéristique pour la division
    best_info_gain = Inf
    best_attribute = ""
    best_threshold = -Inf
    for attribute in attributes
        
        T = copy(tab)
        push!(T, Vector{Any}(missing, size(T,2)))
        # Ajout des branches en construction dans le tableau de branches
        if !isempty(path)
            m = length(path)
            for j in 1:Int((m)/2)
                T[end-1:end, path[2*j-1]] .= path[2*j]
            end
        end
        # Calcul du meilleur seuil
        (threshold, gain_ratio) = Threshold2(data, attribute, target, T)
        if best_info_gain > gain_ratio
            best_info_gain = gain_ratio
            best_attribute = attribute
            best_threshold = threshold
        end
    end

    if isempty(best_attribute)
        # Aucun attribut n'est sélectionné
        majority_class = StatsBase.mode(data[:,target])
        current_path = [path..., majority_class]
        push!(branch, [current_path])
        return majority_class
    end

    # Création du nœud de décision avec la meilleure caractéristique
    tree = Node(best_attribute, Dict())
    depth += 1
    current_path = [path..., best_attribute]

    # Séparation des exemples en fonction du seuil
    left_subset = filter(best_attribute => <=(best_threshold), data)
    right_subset = filter(best_attribute => >(best_threshold), data)
    if isempty(left_subset) || isempty(right_subset)
        return StatsBase.mode(data[:,target])
    else
        value = sort(unique(right_subset[:,best_attribute]))[1]
        tree.children[best_threshold] = MON_C4_5_con(left_subset, target, attributes, [current_path..., best_threshold], branch, R, max_depth, min_samples, depth)
        tree.children[value] = MON_C4_5_con(right_subset, target, attributes, [current_path..., value], branch, R, max_depth, min_samples, depth)
    end
    return tree
end

MON_C4_5_con (generic function with 5 methods)

In [36]:
branch = DataFrame(col=Vector())
tree7 = MON_C4_5(DATA, target, attributes, [], branch, 1)

Node("Taille", Dict{Any, Any}(2 => Node("Qualite", Dict{Any, Any}(2 => 1, 3 => 3, 1 => 1)), 3 => Node("Qualite", Dict{Any, Any}(2 => Node("Prix", Dict{Any, Any}(2 => 3, 1 => 2)), 3 => 3, 1 => 2)), 1 => Node("Qualite", Dict{Any, Any}(2 => 2, 3 => 2, 1 => 1))))

### Rank Tree monotone

In [37]:
function mon_RankTree(data, target, attributes, path, branch, R=1, max_depth=Inf, min_samples=1, depth=0)
    # Vérifier les conditions d'arrêt (profondeur maximale ou nombre minimal d'échantillons)
    if depth >= max_depth || nrow(data) <= min_samples
        current_path = [path..., StatsBase.mode(data[:, target])]
        push!(branch, [current_path])
        # Créer un nœud feuille avec la classe majoritaire dans l'ensemble de données
        return StatsBase.mode(data[:, target])
    end
    # Vérifier si tous les exemples appartiennent à la même classe
    classes = unique(data[:,target])
    if length(classes) == 1
        current_path = [path..., classes[1]]
        push!(branch, [current_path])
        return classes[1]
    end
       
    if !isempty(branch)
        tab = data_tree(branch, attributes, target)
        push!(tab, Vector{Any}(missing, size(tab,2)))
    else
        tab = DataFrame()
        for value in attributes
            tab[!,value] = Vector{Any}(missing, 1)
        end
        tab[!, target] = Vector{Any}(missing, 1)
    end
    
    # Sélectionner l'attribut et la valeur de découpe qui maximisent le score d'ambiguïté totale
    best_attribute = ""
    best_value = 0
    best_score = Inf
    for attribute in attributes
        T = copy(tab)
        push!(T, Vector{Any}(missing, size(T,2)))
        if !isempty(path)
            m = length(path)
            for j in 1:Int((m)/2)
                T[end-1:end, path[2*j-1]] .= path[2*j]
            end
        end
        values = sort(unique(data[:,attribute]))
        n = size(values,1)
        for i in 1:n-1
            
            T[end-1,attribute] = values[i]
            left_child = filter(attribute => <=(values[i]), data)
            T[end-1,target] = StatsBase.mode(left_child[:,target])

            right_child = filter(attribute => >(values[i]), data)
            T[end,attribute] = values[i+1]
            T[end,target] = StatsBase.mode(right_child[:,target])
            
            left_I = ranking_impurity(left_child, target)
            right_I = ranking_impurity(right_child, target)

            INM = NMI(T)
            score = left_I + right_I + R*score_a(INM)
            if best_score > score
                best_attribute = attribute
                best_value = values[i]
                best_score = score
            end
        end
    end
    
    if isempty(best_attribute)
        majority_class = StatsBase.mode(data[:,target])
        current_path = [path..., majority_class]
        push!(branch, [current_path])
        return majority_class
    end

    # Partitionner les exemples en fonction de la valeur de découpe sélectionnée
    left_subset = filter(best_attribute => <=(best_value), data)
    right_subset = filter(best_attribute => >(best_value), data)
    
    # Construction récursive des sous-arbres gauche et droit
    if isempty(left_subset) || isempty(right_subset)
        return StatsBase.mode(data[:,target])
    else
        # Créer le noeud de décision
        node = Node(string(best_attribute), Dict())
        value = sort(unique(right_subset[:,best_attribute]))[1]
        depth += 1
        current_path = [path..., best_attribute]
        node.children[best_value] = mon_RankTree(left_subset, target, attributes, [current_path..., best_value], branch, R, max_depth, min_samples, depth)
        node.children[value] = mon_RankTree(right_subset, target, attributes, [current_path..., value], branch, R, max_depth, min_samples, depth)
    end
    
    return node
end

mon_RankTree (generic function with 5 methods)

In [38]:
branch = DataFrame(col=Vector())
tree8 = mon_RankTree(data_or, target, attributes, [], branch, 1)

Node("Prix", Dict{Any, Any}(2 => Node("Qualite", Dict{Any, Any}(2 => Node("Taille", Dict{Any, Any}(2 => 1, 3 => 2)), 3 => Node("Taille", Dict{Any, Any}(2 => 3, 1 => 2)))), 3 => Node("Qualite", Dict{Any, Any}(2 => Node("Taille", Dict{Any, Any}(3 => 3, 1 => 2)), 1 => Node("Taille", Dict{Any, Any}(2 => 1, 3 => 2))))))

### Forêt aléatoire monotone

In [39]:
function Mon_Forest_RI(data, ntree, n_samples, R_limit, T, target, attributes, max_depth=Inf, min_samples=1)
    M_RF = DataFrame()
    M_RF[!,"tree"] = Vector{Any}(missing, ntree)
    M_RF[!,"INM"] = zeros(ntree)
    n = size(data, 1)
    for i in 1:ntree
        data_2 = bootstrap_sample(data, n_samples) #échantillonnage
        R = rand(1:R_limit) #sélection du facteur d'importance
        # Construction de l'arbre de décision
        M_RF[i,"tree"] = mon_cart(data_2, target, attributes, [], DataFrame(col=Vector()), R, max_depth, min_samples, 0)
        # Extration des branches
        paths = DataFrame(col=Vector())
        branchs(M_RF[i,"tree"], [], paths)
        tab = data_tree(paths, attributes, target)
        # Calcule de l'indice de non-monotonie
        M_RF[i,"INM"] = NMI(tab)
    end
    # Classement par ordre croissant
    sort!(M_RF, "INM")
    # Elagage
    TREE = M_RF[1:Int(round(ntree*T)),:]
    return TREE
end

Mon_Forest_RI (generic function with 3 methods)

In [41]:
forest2 = Mon_Forest_RI(data_or, 15, 5, 5, 0.8, target, attributes)

Row,tree,INM
,Any,Float64
1,"Node(""Prix"", Dict{Any, Any}(4=>3, 3=>2))",0.0
2,"Node(""Prix"", Dict{Any, Any}(4=>Node(""Taille"", Dict{Any, Any}(3=>3, 1=>2)), 3=>2))",0.0
3,"Node(""Taille"", Dict{Any, Any}(3=>Node(""Qualite"", Dict{Any, Any}(2=>3, 1=>2)), 1=>2))",0.0
4,"Node(""Taille"", Dict{Any, Any}(2=>Node(""Taille"", Dict{Any, Any}(2=>3, 1=>2)), 3=>Node(""Qualite"", Dict{Any, Any}(2=>2, 3=>3))))",0.166667
5,"Node(""Qualite"", Dict{Any, Any}(2=>Node(""Qualite"", Dict{Any, Any}(2=>3, 1=>2)), 3=>Node(""Taille"", Dict{Any, Any}(2=>3, 1=>2))))",0.166667
6,"Node(""Taille"", Dict{Any, Any}(2=>Node(""Taille"", Dict{Any, Any}(2=>1, 3=>Node(""Prix"", Dict{Any, Any}(4=>3, 1=>2)))), 1=>Node(""Prix"", Dict{Any, Any}(2=>2, 3=>1))))",0.2
7,"Node(""Prix"", Dict{Any, Any}(2=>Node(""Qualite"", Dict{Any, Any}(3=>3, 1=>1)), 1=>2))",0.333333
8,"Node(""Qualite"", Dict{Any, Any}(2=>3, 1=>Node(""Prix"", Dict{Any, Any}(4=>1, 3=>2))))",0.333333
9,"Node(""Prix"", Dict{Any, Any}(4=>Node(""Qualite"", Dict{Any, Any}(2=>2, 1=>1)), 3=>2))",0.333333


Prévisions

In [42]:
example1 = DataFrame(Taille=2, Qualite=2, Prix=2)
example2 = DataFrame(Taille=2, Qualite=2, Prix=30)
example3 = DataFrame(Taille=2, Qualite=2, Prix=4)
print("C4.5 => ", predict_tree(tree2, example1[1,:]))
print("\nC4.5 monotone => ", predict_tree(tree7, example1[1,:]))
print("\nID3 => ", predict_tree(tree1, example2[1,:]))
print("\nCART => ", predict_bi(tree3, example2[1,:]))
print("\nRanking Tree => ", predict_bi(tree4, example2[1,:]))
print("\nForest_RI => ", predict_rf(forest1, example2[1,:]))
print("\nID3 monotone => ", predict_tree(tree6, example3[1,:]))
print("\nCART monotone => ", predict_bi(tree5, example3[1,:]))
print("\nRanking Tree monotone => ", predict_bi(tree8, example3[1,:]))
print("\nForest_RI monotone => ", predict_rf(forest2, example3[1,:]))

C4.5 => 1
C4.5 monotone => 1
ID3 => 1
CART => 3
Ranking Tree => 1
Forest_RI => 3
ID3 monotone => 1
CART monotone => 3
Ranking Tree monotone => 3
Forest_RI monotone => 3

# Applications

## Mesures d'évaluation

Le tau de Kendall (modifié) $$\frac{\text{nombre de paires concordantes}}{n*(n-1)}$$

In [99]:
function kendall_tau(r1, r2)
    n = length(r1)
    if n < 2
        return 1
    end
    tau = 0.0
    for i = 1:n-1
        for j=i+1:n
            if (r1[i]-r1[j]) * (r2[i]-r2[j]) > 0
                tau += 1
            elseif r1[i]-r1[j]==0 && r2[i]-r2[j]==0
                tau = tau
            else
                tau -= 1
            end
        end
    end
    2 * tau / (n * (n-1))
end

kendall_tau (generic function with 1 method)

$$$$L'erreur 0-1 moyenne (mean zero-one error (MZE))
$$\text{MZE} = \frac{1}{N} \displaystyle\sum _{i=1}^N 1_{\{y_i^* \neq y_i\}}$$


In [44]:
function MZE(X, Y)
    n=length(X)
    r = 0
    for i in 1:n
        if X[i] != Y[i]
            r += 1
        end
    end
    return r/n
end

MZE (generic function with 1 method)

$$$$L'erreur absolue moyenne (mean absolute error (MAE)) : 
$$\text{MAE} = \frac{1}{N} \displaystyle\sum _{i=1}^N | \mathcal{O}(y_i) - \mathcal{O}(y_i^*)|$$


In [45]:
function MAE(X, Y)
    n=length(X)
    r = 0
    for i in 1:n
        r += abs(X[i]-Y[i])
    end
    return r/n
end

MAE (generic function with 1 method)

## Données

### Car evaluation

In [46]:
d = readdlm("car.data", ',')
    
data1 = DataFrame()
data1[!,"Buying"] = replace!(d[:,1], "vhigh" => 1, "high" => 2, "med" => 3, "low" => 4)
data1[!,"Maint"] = replace!(d[:,2], "vhigh" => 1, "high" => 2, "med" => 3, "low" => 4)
data1[!,"Doors"] = replace!(d[:,3], 2 => 1, 3 => 2, 4 => 3, "5more" => 4, )
data1[!,"Persons"] = replace!(d[:,4], 2 => 1, 4 => 2, "more" => 3)
data1[!,"Lug_boot"] = replace!(d[:,5], "small" => 1, "med" => 2, "big" => 3)
data1[!,"Safety"] = replace!(d[:,6], "low" => 1, "med" => 2, "high" => 3)
data1[!,"Acceptability"] = replace!(d[:,7], "unacc" => 1, "acc" => 2, "good" => 3, "vgood" => 4)

NMI(data1)

5.629543846104356e-5

In [47]:
attributes1 = names(data1)[1:end-1]
target1 = names(data1)[end];

In [48]:
d3 = readdlm("yeast.data")

data3 = DataFrame()
data3[!,"mcg"] = d3[:,2]
data3[!,"gvh"] = d3[:,3]
data3[!,"alm"] = d3[:,4]
data3[!,"mit"] = d3[:,5]
data3[!,"erl"] = d3[:,6]
data3[!,"pox"] = d3[:,7]
data3[!,"vac"] = d3[:,8]
data3[!,"nuc"] = d3[:,9]
data3[!,"class"] = replace!(d3[:,10], "CYT" => 1, "NUC" => 2, "MIT" => 3, "ME3" => 4, "ME2" => 5, "ME1" => 6, "EXC" =>7, "VAC" =>8, "POX" => 9, "ERL" => 10)

NMI(data3)

0.02075726154276772

In [49]:
attributes3 = names(data3)[1:end-1]
target3 = names(data3)[end];

### Segmentation

In [50]:
d2 = readdlm("segment.dat", ',')

data2 = DataFrame()
data2[!,"R_col"] = d2[:,1]
data2[!,"R_row"] = d2[:,2]
data2[!,"Pixel_count"] = d2[:,3]
data2[!,"Density_5"] = d2[:,4]
data2[!,"Density_2"] = d2[:,5]
data2[!,"Vedge_mean"] = d2[:,6]
data2[!,"Vegde_sd"] = d2[:,7]
data2[!,"Hedge_mean"] = d2[:,8]
data2[!,"Hedge_sd"] = d2[:,9]
data2[!,"Intensity"] = d2[:,10]
data2[!,"Rawred"] = d2[:,11]
data2[!,"Rawblue"] = d2[:,12]
data2[!,"Rawgreen"] = d2[:,13]
data2[!,"Exred"] = d2[:,14]
data2[!,"Exblue"] = d2[:,15]
data2[!,"Exgreen"] = d2[:,16]
data2[!,"Value"] = d2[:,17]
data2[!,"Saturatoin"] = d2[:,18]
data2[!,"Hue"] = d2[:,19]
data2[!,"Classe"] = Int.(d2[:,20])

NMI(data2)

3.7496789337412986e-7

$$$$Remplacement des classes pour rendre les données monotones

In [51]:
data2[1429,end] = 5
data2[812,end] = 3
NMI(data2)

0.0

In [52]:
attributes2 = names(data2)[1:end-1]
target2 = names(data2)[end];

### Paramètres

In [53]:
min_s = 2 #nombre minimum de d'exemples pour effectuer un partitionnement
max_d = 90 #profondeur maximale de l'arbre
R=1 #facteur d'importance du score d'ambiguïté de l'ordre
R_max = 100 #limite des facteurs d'importance du score d'ambiguïté de l'ordre pour les forêts aléatoires monotones
T = 0.5 #seuil d'élagage des forêts aléatoires
ntrees = 100 #nombre d'arbres pour les forêts aléatoires
n_samples1 = Int(round(10*size(data1,1)*0.7/ntrees)) #taille de l'échantillon bootstrap
n_samples2 = Int(round(10*size(data2,1)*0.7/ntrees)) #taille de l'échantillon bootstrap
n_samples3 = Int(round(10*size(data3,1)*0.7/ntrees)) #taille de l'échantillon bootstrap
n_split = 0.7 #portion de données d'entrainement
n_exp = 20; #nombre de répétitions

## Fonctions pour l'évalution des performances

In [54]:
# arbre de décision
function evaluation(TREE, don, n_exp, attributes, target)
    EVAL = DataFrame()
    EVAL[!,"INM"] = zeros(n_exp)
    EVAL[!,"MAE"] = zeros(n_exp)
    EVAL[!,"MZE"] = zeros(n_exp)
    EVAL[!,"leaves"] = zeros(n_exp)
    EVAL[!,"tau"] = zeros(n_exp)
    for i in 1:n_exp
        EVAL[i,"tau"] = kendall_tau(don[i,"test"][:,end], TREE[i,"pred"])
        EVAL[i,"MAE"] = MAE(don[i,"test"][:,end], TREE[i,"pred"])
        EVAL[i,"MZE"] = MZE(don[i,"test"][:,end], TREE[i,"pred"])
    
        paths = DataFrame(col=Vector())
        branchs(TREE[i,"tree"], [], paths)
        tab = data_tree(paths, attributes, target)
        EVAL[i,"leaves"] = size(tab,1)
    
        D = copy(don[i,"test"])
        D[:,end] = TREE[i,"pred"]
    
        EVAL[i,"INM"] = NMI(D)
    end
    return EVAL
end

# forêt
function evaluation2(TREE, don, n_exp, attributes, target)
    EVAL = DataFrame()
    EVAL[!,"INM"] = zeros(n_exp)
    EVAL[!,"MAE"] = zeros(n_exp)
    EVAL[!,"MZE"] = zeros(n_exp)
    EVAL[!,"leaves"] = zeros(n_exp)
    EVAL[!,"tau"] = zeros(n_exp)
    for i in 1:n_exp
        EVAL[i,"tau"] = kendall_tau(don[i,"test"][:,end], TREE[i,"pred"])
        EVAL[i,"MAE"] = MAE(don[i,"test"][:,end], TREE[i,"pred"])
        EVAL[i,"MZE"] = MZE(don[i,"test"][:,end], TREE[i,"pred"])
        
        n = size(TREE[i,"tree"],1)
        l = 0
        for j in 1:n
            paths = DataFrame(col=Vector())
            branchs(TREE[i,"tree"][j,1], [], paths)
            tab = data_tree(paths, attributes, target)
            l += size(tab,1)
        end
            
        EVAL[i,"leaves"] = l/n
    
        D = copy(don[i,"test"])
        D[:,end] = TREE[i,"pred"]
    
        EVAL[i,"INM"] = NMI(D)
    end
    return EVAL
end


evaluation2 (generic function with 1 method)

## Temps d'exécution

In [165]:
print("INM : ")
@btime NMI(data1)

print("Entropie : ")
@btime entropy(data1, target1)

print("Indice de Gini : ")
@btime gini_index(data1, target1)

print("Ranking impurity : ")
@btime ranking_impurity(data1, target1),

INM :   10.379 s (78937593 allocations: 1.18 GiB)
Entropie :   161.600 μs (3192 allocations: 120.58 KiB)
Indice de Gini :   166.800 μs (3192 allocations: 120.56 KiB)
Ranking impurity :   725.700 μs (3251 allocations: 237.91 KiB)


948471.0

In [152]:
print("ID3 : ")
@btime ID3(data1, attributes1, target1)

print("ID3 monotone : ")
@btime MON_ID3(data1, target1, attributes1, [], DataFrame(col=Vector()), R)

print("C4.5 : ")
@btime C4_5(data1, target1, attributes1)

print("C4.5 monotone : ")
@btime MON_C4_5(data1, target1, attributes1, [], DataFrame(col=Vector()), R)

print("Ranking Tree : ")
@btime RankTree(data1, target1, attributes1)

print("Ranking Tree monotone : ")
@btime mon_RankTree(data1, target1, attributes1, [], DataFrame(col=Vector()), R)

print("CART : ")
@btime cart(data1, target1, attributes1)

print("CART monotone : ")
@btime mon_cart(data1, target1, attributes1, [], DataFrame(col=Vector()), R)

print("Forêt aléatoire : ")
@btime Forest_RI(data1, ntrees, n_samples1, attributes1, target1)

print("Forêt aléatoire monotone :")
@btime Mon_Forest_RI(data1, ntrees, n_samples1, R_max, T, target1, attributes1);

ID3 :   24.699 ms (375280 allocations: 19.15 MiB)
ID3 monotone :   3.007 s (536772 allocations: 36.37 MiB)
C4.5 :   36.608 ms (524535 allocations: 27.23 MiB)
C4.5 monotone :   3.117 s (642175 allocations: 42.46 MiB)
Ranking Tree :   56.128 ms (674019 allocations: 41.10 MiB)
Ranking Tree monotone :   870.305 ms (787488 allocations: 55.91 MiB)
CART :   69.149 ms (953765 allocations: 50.12 MiB)
CART monotone :   428.615 ms (622891 allocations: 37.87 MiB)
Forêt aléatoire :   2.488 s (22260941 allocations: 1.29 GiB)
Forêt aléatoire monotone :  3.857 s (15005539 allocations: 968.60 MiB)


In [153]:
print("ID3 : ")
@btime ID3(data2, attributes2, target2)

print("ID3 monotone : ")
@btime MON_ID3(data2, target2, attributes2, [], DataFrame(col=Vector()), R)

print("C4.5 : ")
@btime C4_5_con(data2, target2, attributes2)

print("C4.5 monotone : ")
@btime MON_C4_5_con(data2, target2, attributes2, [], DataFrame(col=Vector()), R)

print("Ranking Tree : ")
@btime RankTree(data2, target2, attributes2)

print("Ranking Tree monotone : ")
@btime mon_RankTree(data2, target2, attributes2, [], DataFrame(col=Vector()), R)

print("CART : ")
@btime cart(data2, target2, attributes2)

print("CART monotone : ")
@btime mon_cart(data2, target2, attributes2, [], DataFrame(col=Vector()), R)

print("Forêt aléatoire : ")
@btime Forest_RI(data2, ntrees, n_samples2, attributes2, target2)

print("Forêt aléatoire monotone :")
@btime Mon_Forest_RI(data2, ntrees, n_samples2, R_max, T, target2, attributes2);

ID3 :   678.467 ms (5496331 allocations: 426.53 MiB)
ID3 monotone :   112.640 s (2815267175 allocations: 42.35 GiB)
C4.5 :   12.706 s (74371467 allocations: 21.14 GiB)
C4.5 monotone :   29.698 s (76304503 allocations: 33.95 GiB)
Ranking Tree :   16.634 s (75061967 allocations: 30.21 GiB)
Ranking Tree monotone :   208.370 s (115671754 allocations: 63.85 GiB)
CART :   14.680 s (78985538 allocations: 24.06 GiB)
CART monotone :   209.125 s (104387528 allocations: 38.48 GiB)
Forêt aléatoire :   131.351 s (871229524 allocations: 84.85 GiB)
Forêt aléatoire monotone :  3816.023 s (1504063719 allocations: 143.94 GiB)


In [72]:
print("ID3 : ")
@btime ID3(data3, attributes3, target3)

print("ID3 monotone : ")
@btime MON_ID3(data3, target3, attributes3, [], DataFrame(col=Vector()), R)

print("C4.5 : ")
@btime C4_5_con(data3, target3, attributes3)

print("C4.5 monotone : ")
@btime MON_C4_5_con(data3, target3, attributes3, [], DataFrame(col=Vector()), R)

print("Ranking Tree : ")
@btime RankTree(data3, target3, attributes3)

print("Ranking Tree monotone : ")
@btime mon_RankTree(data3, target3, attributes3, [], DataFrame(col=Vector()), R)

print("CART : ")
@btime cart(data3, target3, attributes3)

print("CART monotone : ")
@btime mon_cart(data3, target3, attributes3, [], DataFrame(col=Vector()), R)

print("Forêt aléatoire : ")
@btime Forest_RI(data3, ntrees, n_samples3, attributes3, target3)

print("Forêt aléatoire monotone :")
@btime Mon_Forest_RI(data3, ntrees, n_samples3, R_max, T, target3, attributes3);

ID3 :   271.209 ms (2894569 allocations: 140.61 MiB)
ID3 monotone :   305.745 s (6502092087 allocations: 97.21 GiB)
C4.5 :   3.006 s (29383490 allocations: 1.63 GiB)
C4.5 monotone :   1415.348 s (5332805523 allocations: 81.11 GiB)
Ranking Tree :   2.868 s (20354156 allocations: 1.33 GiB)
Ranking Tree monotone :   1508.894 s (9232276323 allocations: 139.21 GiB)
CART :   2.797 s (26267553 allocations: 1.49 GiB)
CART monotone :   1556.592 s (32960678 allocations: 2.01 GiB)
Forêt aléatoire :   18.202 s (137918127 allocations: 8.33 GiB)
Forêt aléatoire monotone :  116.427 s (182189916 allocations: 11.38 GiB)


## Expériences

### Car evaluation

In [93]:
don = DataFrame()
don[!,"train"] = Vector{Any}(missing, n_exp)
don[!,"test"] = Vector{Any}(missing, n_exp)

# ID3
TREE1 = DataFrame()
TREE1[!,"tree"] = Vector{Any}(missing, n_exp)
TREE1[!,"pred"] = Vector{Any}(missing, n_exp)

# C4.5
TREE2 = DataFrame()
TREE2[!,"tree"] = Vector{Any}(missing, n_exp)
TREE2[!,"pred"] = Vector{Any}(missing, n_exp)

# CART
TREE3 = DataFrame()
TREE3[!,"tree"] = Vector{Any}(missing, n_exp)
TREE3[!,"pred"] = Vector{Any}(missing, n_exp)

# Ranking Tree
TREE4 = DataFrame()
TREE4[!,"tree"] = Vector{Any}(missing, n_exp)
TREE4[!,"pred"] = Vector{Any}(missing, n_exp)

# ID3 monotone
TREE5 = DataFrame()
TREE5[!,"tree"] = Vector{Any}(missing, n_exp)
TREE5[!,"pred"] = Vector{Any}(missing, n_exp)

# C4.5 monotone
TREE6 = DataFrame()
TREE6[!,"tree"] = Vector{Any}(missing, n_exp)
TREE6[!,"pred"] = Vector{Any}(missing, n_exp)

# CART monotone
TREE7 = DataFrame()
TREE7[!,"tree"] = Vector{Any}(missing, n_exp)
TREE7[!,"pred"] = Vector{Any}(missing, n_exp)

# Ranking Tree monotone
TREE8 = DataFrame()
TREE8[!,"tree"] = Vector{Any}(missing, n_exp)
TREE8[!,"pred"] = Vector{Any}(missing, n_exp)

# forêt aléatoire
FOREST1 = DataFrame()
FOREST1[!,"tree"] = Vector{Any}(missing, n_exp)
FOREST1[!,"pred"] = Vector{Any}(missing, n_exp)

# forêt aléatoire monotone
FOREST2 = DataFrame()
FOREST2[!,"tree"] = Vector{Any}(missing, n_exp)
FOREST2[!,"pred"] = Vector{Any}(missing, n_exp)

for i in 1:n_exp
    (don[i,"train"], don[i,"test"]) =  partition(data1, n_split, shuffle=true)
    TREE1[i,"tree"] = ID3(don[i,"train"], attributes1, target1)
    TREE2[i,"tree"] = C4_5(don[i,"train"], target1, attributes1, max_d, min_s)
    TREE3[i,"tree"] = cart(don[i,"train"], target1, attributes1, max_d, min_s)
    TREE4[i,"tree"] = RankTree(don[i,"train"], target1, attributes1, max_d, min_s)
    TREE5[i,"tree"] = MON_ID3(don[i,"train"], target1, attributes1, [], DataFrame(col=Vector()), R, max_d, min_s)
    TREE6[i,"tree"] = MON_C4_5(don[i,"train"], target1, attributes1, [], DataFrame(col=Vector()), R, max_d, min_s)
    TREE7[i,"tree"] = mon_cart(don[i,"train"], target1, attributes1, [], DataFrame(col=Vector()), R, max_d, min_s)
    TREE8[i,"tree"] = mon_RankTree(don[i,"train"], target1, attributes1, [], DataFrame(col=Vector()), R, max_d, min_s)
    FOREST1[i,"tree"] = Forest_RI(don[i,"train"], ntrees, n_samples1, attributes1, target1)
    FOREST2[i,"tree"] = Mon_Forest_RI(don[i,"train"], ntrees, n_samples1, R_max, T, target1, attributes1, max_d, min_s)
    
    n = size(don[i,"test"],1)
    TREE1[i,"pred"] = zeros(Int,n)
    TREE2[i,"pred"] = zeros(Int,n)
    TREE3[i,"pred"] = zeros(Int,n)
    TREE4[i,"pred"] = zeros(Int,n)
    TREE5[i,"pred"] = zeros(Int,n)
    TREE6[i,"pred"] = zeros(Int,n)
    TREE7[i,"pred"] = zeros(Int,n)
    TREE8[i,"pred"] = zeros(Int,n)
    FOREST1[i,"pred"] = zeros(Int,n)
    FOREST2[i,"pred"] = zeros(Int,n)
    for j in 1:n
        TREE1[i,"pred"][j] = predict_tree(TREE1[i,"tree"], don[i,"test"][j,1:end-1])
        TREE2[i,"pred"][j] = predict_tree(TREE2[i,"tree"], don[i,"test"][j,1:end-1])
        TREE3[i,"pred"][j] = predict_bi(TREE3[i,"tree"], don[i,"test"][j,1:end-1])
        TREE4[i,"pred"][j] = predict_bi(TREE4[i,"tree"], don[i,"test"][j,1:end-1])
        TREE5[i,"pred"][j] = predict_tree(TREE5[i,"tree"], don[i,"test"][j,1:end-1])
        TREE6[i,"pred"][j] = predict_tree(TREE6[i,"tree"], don[i,"test"][j,1:end-1])
        TREE7[i,"pred"][j] = predict_bi(TREE7[i,"tree"], don[i,"test"][j,1:end-1])
        TREE8[i,"pred"][j] = predict_bi(TREE8[i,"tree"], don[i,"test"][j,1:end-1])
        FOREST1[i,"pred"][j] = predict_rf(FOREST1[i,"tree"], don[i,"test"][j,1:end-1])
        FOREST2[i,"pred"][j] = predict_rf(FOREST2[i,"tree"], don[i,"test"][j,1:end-1])
    end
end

TREE = Dict()
TREE["ID3"] = TREE1
TREE["C45"] = TREE2
TREE["CART"] = TREE3
TREE["RankTree"] = TREE4
TREE["MON_ID3"] = TREE5
TREE["MON_C45"] = TREE6
TREE["MON_CART"] = TREE7
TREE["MON_RankTree"] = TREE8
TREE["FOREST1"] = FOREST1
TREE["FOREST2"] = FOREST2;

In [84]:
for j in 1:n_exp
    CSV.write(string("Données1_Train",j,".csv"), don[j,"train"])
    CSV.write(string("Données1_Test",j,".csv"), don[j,"test"])
    for i in keys(TREE)
        D = copy(don[j,"test"])
        D[:,end] = TREE[i][j,"pred"]
        CSV.write(string("Données1_",i,"_Pred",j,".csv"), D)
    end
end
for i in keys(TREE)
    if i!="FOREST2" && i!="FOREST1"
        EVAL = evaluation(TREE[i], don, n_exp, attributes1, target1)
    else
        EVAL = evaluation2(TREE[i], don, n_exp, attributes1, target1)
    end
    CSV.write(string("Données1_",i,"_Eval.csv"), EVAL)
end

### Segmentation

In [158]:
don = DataFrame()
don[!,"train"] = Vector{Any}(missing, n_exp)
don[!,"test"] = Vector{Any}(missing, n_exp)

# ID3
TREE1 = DataFrame()
TREE1[!,"tree"] = Vector{Any}(missing, n_exp)
TREE1[!,"pred"] = Vector{Any}(missing, n_exp)

# C4.5
TREE2 = DataFrame()
TREE2[!,"tree"] = Vector{Any}(missing, n_exp)
TREE2[!,"pred"] = Vector{Any}(missing, n_exp)

# CART
TREE3 = DataFrame()
TREE3[!,"tree"] = Vector{Any}(missing, n_exp)
TREE3[!,"pred"] = Vector{Any}(missing, n_exp)

# Ranking Tree
TREE4 = DataFrame()
TREE4[!,"tree"] = Vector{Any}(missing, n_exp)
TREE4[!,"pred"] = Vector{Any}(missing, n_exp)

# ID3 monotone
TREE5 = DataFrame()
TREE5[!,"tree"] = Vector{Any}(missing, n_exp)
TREE5[!,"pred"] = Vector{Any}(missing, n_exp)

# C4.5 monotone
TREE6 = DataFrame()
TREE6[!,"tree"] = Vector{Any}(missing, n_exp)
TREE6[!,"pred"] = Vector{Any}(missing, n_exp)

# CART monotone
TREE7 = DataFrame()
TREE7[!,"tree"] = Vector{Any}(missing, n_exp)
TREE7[!,"pred"] = Vector{Any}(missing, n_exp)

# Ranking Tree monotone
TREE8 = DataFrame()
TREE8[!,"tree"] = Vector{Any}(missing, n_exp)
TREE8[!,"pred"] = Vector{Any}(missing, n_exp)

# forêt aléatoire
FOREST1 = DataFrame()
FOREST1[!,"tree"] = Vector{Any}(missing, n_exp)
FOREST1[!,"pred"] = Vector{Any}(missing, n_exp)

# forêt aléatoire monotone
FOREST2 = DataFrame()
FOREST2[!,"tree"] = Vector{Any}(missing, n_exp)
FOREST2[!,"pred"] = Vector{Any}(missing, n_exp)

for i in 1:n_exp
    (don[i,"train"], don[i,"test"]) =  partition(data2, n_split, shuffle=true)
    TREE1[i,"tree"] = ID3(don[i,"train"], attributes2, target2)
    TREE2[i,"tree"] = C4_5_con(don[i,"train"], target2, attributes2, max_d, min_s)
    TREE3[i,"tree"] = cart(don[i,"train"], target2, attributes2, max_d, min_s)
    TREE4[i,"tree"] = RankTree(don[i,"train"], target2, attributes2, max_d, min_s)
    TREE5[i,"tree"] = MON_ID3(don[i,"train"], target2, attributes2, [], DataFrame(col=Vector()), R, max_d, min_s)
    TREE6[i,"tree"] = MON_C4_5_con(don[i,"train"], target2, attributes2, [], DataFrame(col=Vector()), R, max_d, min_s)
    TREE7[i,"tree"] = mon_cart(don[i,"train"], target2, attributes2, [], DataFrame(col=Vector()), R, max_d, min_s)
    TREE8[i,"tree"] = mon_RankTree(don[i,"train"], target2, attributes2, [], DataFrame(col=Vector()), R, max_d, min_s)
    FOREST1[i,"tree"] = Forest_RI(don[i,"train"], ntrees, n_samples2, attributes2, target2)
    FOREST2[i,"tree"] = Mon_Forest_RI(don[i,"train"], ntrees, n_samples2, R_max, T, target2, attributes2, max_d, min_s)
    
    n = size(don[i,"test"],1)
    TREE1[i,"pred"] = zeros(Int,n)
    TREE2[i,"pred"] = zeros(Int,n)
    TREE3[i,"pred"] = zeros(Int,n)
    TREE4[i,"pred"] = zeros(Int,n)
    TREE5[i,"pred"] = zeros(Int,n)
    TREE6[i,"pred"] = zeros(Int,n)
    TREE7[i,"pred"] = zeros(Int,n)
    TREE8[i,"pred"] = zeros(Int,n)
    FOREST1[i,"pred"] = zeros(Int,n)
    FOREST2[i,"pred"] = zeros(Int,n)
    for j in 1:n
        TREE1[i,"pred"][j] = predict_tree(TREE1[i,"tree"], don[i,"test"][j,1:end-1])
        TREE2[i,"pred"][j] = predict_bi(TREE2[i,"tree"], don[i,"test"][j,1:end-1])
        TREE3[i,"pred"][j] = predict_bi(TREE3[i,"tree"], don[i,"test"][j,1:end-1])
        TREE4[i,"pred"][j] = predict_bi(TREE4[i,"tree"], don[i,"test"][j,1:end-1])
        TREE5[i,"pred"][j] = predict_tree(TREE5[i,"tree"], don[i,"test"][j,1:end-1])
        TREE6[i,"pred"][j] = predict_bi(TREE6[i,"tree"], don[i,"test"][j,1:end-1])
        TREE7[i,"pred"][j] = predict_bi(TREE7[i,"tree"], don[i,"test"][j,1:end-1])
        TREE8[i,"pred"][j] = predict_bi(TREE8[i,"tree"], don[i,"test"][j,1:end-1])
        FOREST1[i,"pred"][j] = predict_rf(FOREST1[i,"tree"], don[i,"test"][j,1:end-1])
        FOREST2[i,"pred"][j] = predict_rf(FOREST2[i,"tree"], don[i,"test"][j,1:end-1])
    end
end

TREE = Dict()
TREE["ID3"] = TREE1
TREE["C45"] = TREE2
TREE["CART"] = TREE3
TREE["RankTree"] = TREE4
TREE["MON_ID3"] = TREE5
TREE["MON_C45"] = TREE6
TREE["MON_CART"] = TREE7
TREE["MON_RankTree"] = TREE8
TREE["FOREST1"] = FOREST1
TREE["FOREST2"] = FOREST2;



In [159]:
for j in 1:n_exp
    CSV.write(string("Données2_Train",j,".csv"), don[j,"train"])
    CSV.write(string("Données2_Test",j,".csv"), don[j,"test"])
    for i in keys(TREE)
        D = copy(don[j,"test"])
        D[:,end] = TREE[i][j,"pred"]
        CSV.write(string("Données2_",i,"_Pred",j,".csv"), D)
    end
end
for i in keys(TREE)
    if i!="FOREST2" && i!="FOREST1"
        EVAL = evaluation(TREE[i], don, n_exp, attributes2, target2)
    else
        EVAL = evaluation2(TREE[i], don, n_exp, attributes2, target2)
    end
    CSV.write(string("Données2_",i,"_Eval.csv"), EVAL)
end

### Yeast

In [58]:
don = DataFrame()
don[!,"train"] = Vector{Any}(missing, n_exp)
don[!,"test"] = Vector{Any}(missing, n_exp)

# ID3
TREE1 = DataFrame()
TREE1[!,"tree"] = Vector{Any}(missing, n_exp)
TREE1[!,"pred"] = Vector{Any}(missing, n_exp)

# C4.5
TREE2 = DataFrame()
TREE2[!,"tree"] = Vector{Any}(missing, n_exp)
TREE2[!,"pred"] = Vector{Any}(missing, n_exp)

# CART
TREE3 = DataFrame()
TREE3[!,"tree"] = Vector{Any}(missing, n_exp)
TREE3[!,"pred"] = Vector{Any}(missing, n_exp)

# Ranking Tree
TREE4 = DataFrame()
TREE4[!,"tree"] = Vector{Any}(missing, n_exp)
TREE4[!,"pred"] = Vector{Any}(missing, n_exp)

# ID3 monotone
TREE5 = DataFrame()
TREE5[!,"tree"] = Vector{Any}(missing, n_exp)
TREE5[!,"pred"] = Vector{Any}(missing, n_exp)

# C4.5 monotone
TREE6 = DataFrame()
TREE6[!,"tree"] = Vector{Any}(missing, n_exp)
TREE6[!,"pred"] = Vector{Any}(missing, n_exp)

# CART monotone
TREE7 = DataFrame()
TREE7[!,"tree"] = Vector{Any}(missing, n_exp)
TREE7[!,"pred"] = Vector{Any}(missing, n_exp)

# Ranking Tree monotone
TREE8 = DataFrame()
TREE8[!,"tree"] = Vector{Any}(missing, n_exp)
TREE8[!,"pred"] = Vector{Any}(missing, n_exp)

# forêt aléatoire
FOREST1 = DataFrame()
FOREST1[!,"tree"] = Vector{Any}(missing, n_exp)
FOREST1[!,"pred"] = Vector{Any}(missing, n_exp)

# forêt aléatoire monotone
FOREST2 = DataFrame()
FOREST2[!,"tree"] = Vector{Any}(missing, n_exp)
FOREST2[!,"pred"] = Vector{Any}(missing, n_exp)

for i in 1:n_exp
    (don[i,"train"], don[i,"test"]) =  partition(data3, n_split, shuffle=true)
    TREE1[i,"tree"] = ID3(don[i,"train"], attributes3, target3)
    TREE2[i,"tree"] = C4_5_con(don[i,"train"], target3, attributes3, max_d, min_s)
    TREE3[i,"tree"] = cart(don[i,"train"], target3, attributes3, max_d, min_s)
    TREE4[i,"tree"] = RankTree(don[i,"train"], target3, attributes3, max_d, min_s)
    TREE5[i,"tree"] = MON_ID3(don[i,"train"], target3, attributes3, [], DataFrame(col=Vector()), R, max_d, min_s)
    TREE6[i,"tree"] = MON_C4_5_con(don[i,"train"], target3, attributes3, [], DataFrame(col=Vector()), R, max_d, min_s)
    TREE7[i,"tree"] = mon_cart(don[i,"train"], target3, attributes3, [], DataFrame(col=Vector()), R, max_d, min_s)
    TREE8[i,"tree"] = mon_RankTree(don[i,"train"], target3, attributes3, [], DataFrame(col=Vector()), R, max_d, min_s)
    FOREST1[i,"tree"] = Forest_RI(don[i,"train"], ntrees, n_samples3, attributes3, target3)
    FOREST2[i,"tree"] = Mon_Forest_RI(don[i,"train"], ntrees, n_samples3, R_max, T, target3, attributes3, max_d, min_s)
    
    n = size(don[i,"test"],1)
    TREE1[i,"pred"] = zeros(Int,n)
    TREE2[i,"pred"] = zeros(Int,n)
    TREE3[i,"pred"] = zeros(Int,n)
    TREE4[i,"pred"] = zeros(Int,n)
    TREE5[i,"pred"] = zeros(Int,n)
    TREE6[i,"pred"] = zeros(Int,n)
    TREE7[i,"pred"] = zeros(Int,n)
    TREE8[i,"pred"] = zeros(Int,n)
    FOREST1[i,"pred"] = zeros(Int,n)
    FOREST2[i,"pred"] = zeros(Int,n)
    for j in 1:n
        TREE1[i,"pred"][j] = predict_tree(TREE1[i,"tree"], don[i,"test"][j,1:end-1])
        TREE2[i,"pred"][j] = predict_bi(TREE2[i,"tree"], don[i,"test"][j,1:end-1])
        TREE3[i,"pred"][j] = predict_bi(TREE3[i,"tree"], don[i,"test"][j,1:end-1])
        TREE4[i,"pred"][j] = predict_bi(TREE4[i,"tree"], don[i,"test"][j,1:end-1])
        TREE5[i,"pred"][j] = predict_tree(TREE5[i,"tree"], don[i,"test"][j,1:end-1])
        TREE6[i,"pred"][j] = predict_bi(TREE6[i,"tree"], don[i,"test"][j,1:end-1])
        TREE7[i,"pred"][j] = predict_bi(TREE7[i,"tree"], don[i,"test"][j,1:end-1])
        TREE8[i,"pred"][j] = predict_bi(TREE8[i,"tree"], don[i,"test"][j,1:end-1])
        FOREST1[i,"pred"][j] = predict_rf(FOREST1[i,"tree"], don[i,"test"][j,1:end-1])
        FOREST2[i,"pred"][j] = predict_rf(FOREST2[i,"tree"], don[i,"test"][j,1:end-1])
    end
end

TREE = Dict()
TREE["ID3"] = TREE1
TREE["C45"] = TREE2
TREE["CART"] = TREE3
TREE["RankTree"] = TREE4
TREE["MON_ID3"] = TREE5
TREE["MON_C45"] = TREE6
TREE["MON_CART"] = TREE7
TREE["MON_RankTree"] = TREE8
TREE["FOREST1"] = FOREST1
TREE["FOREST2"] = FOREST2;

In [60]:
for j in 1:n_exp
    CSV.write(string("Données3_Train",j,".csv"), don[j,"train"])
    CSV.write(string("Données3_Test",j,".csv"), don[j,"test"])
    for i in keys(TREE)
        D = copy(don[j,"test"])
        D[:,end] = TREE[i][j,"pred"]
        CSV.write(string("Données3_",i,"_Pred",j,".csv"), D)
    end
end
for i in keys(TREE)
    if i!="FOREST2" && i!="FOREST1"
        EVAL = evaluation(TREE[i], don, n_exp, attributes3, target3)
    else
        EVAL = evaluation2(TREE[i], don, n_exp, attributes3, target3)
    end
    CSV.write(string("Données3_",i,"_Eval.csv"), EVAL)
end

## Résultats

In [94]:
resultat1 = DataFrame()
resultat1[!,"Methode"] = Vector{String}(undef, 10)
resultat1[!,"INM"] = zeros(10)
resultat1[!,"MAE"] = zeros(10)
resultat1[!,"MZE"] = zeros(10)
resultat1[!,"leaves"] = zeros(10)
resultat1[!,"tau"] = zeros(10)

resultat2 = DataFrame()
resultat2[!,"Methode"] = Vector{String}(undef, 10)
resultat2[!,"INM"] = zeros(10)
resultat2[!,"MAE"] = zeros(10)
resultat2[!,"MZE"] = zeros(10)
resultat2[!,"leaves"] = zeros(10)
resultat2[!,"tau"] = zeros(10)

resultat3 = DataFrame()
resultat3[!,"Methode"] = Vector{String}(undef, 10)
resultat3[!,"INM"] = zeros(10)
resultat3[!,"MAE"] = zeros(10)
resultat3[!,"MZE"] = zeros(10)
resultat3[!,"leaves"] = zeros(10)
resultat3[!,"tau"] = zeros(10)

k=0
for i in keys(TREE)
    k+=1
    D = CSV.read(string("Données1_",i,"_Eval.csv"), DataFrame)
    resultat1[k,"Methode"] = i
    for j in names(D)
        resultat1[k,j] = mean(D[:,j])
    end
    
    D = CSV.read(string("Données2_",i,"_Eval.csv"), DataFrame)
    resultat2[k,"Methode"] = i
    for j in names(D)
        resultat2[k,j] = mean(D[:,j])
    end
    
    D = CSV.read(string("Données3_",i,"_Eval.csv"), DataFrame)
    resultat3[k,"Methode"] = i
    for j in names(D)
        resultat3[k,j] = mean(D[:,j])
    end
    
end

### Car

In [189]:
resultat1

Row,Methode,INM,MAE,MZE,leaves,tau
,String,Float64,Float64,Float64,Float64,Float64
1,MON_ID3,0.000657939,0.0906371,0.0784749,204.15,0.799906
2,MON_C45,0.000568322,0.0867761,0.075,203.45,0.806494
3,CART,0.000130318,0.0315637,0.027027,66.5,0.931979
4,MON_RankTree,0.000169526,0.0374517,0.0316602,77.9,0.913924
5,FOREST2,1.12021e-6,0.0660232,0.057529,18.738,0.860002
6,ID3,0.000561601,0.0868726,0.0758687,214.45,0.805094
7,RankTree,0.000181475,0.03861,0.032722,77.8,0.911709
8,C45,0.000586619,0.0871622,0.0755792,204.25,0.807633
9,MON_CART,0.00012397,0.0305985,0.0260618,66.7,0.93537


### Segmentation

In [171]:
resultat2

Row,Methode,INM,MAE,MZE,leaves,tau
,String,Float64,Float64,Float64,Float64,Float64
1,MON_ID3,1.87674e-6,0.647547,0.360462,1479.6,0.555126
2,MON_C45,6.25579e-7,0.0965368,0.0484127,68.4,0.920794
3,CART,2.08526e-6,0.0821789,0.0450938,58.1,0.930689
4,MON_RankTree,2.50231e-6,0.116883,0.058153,70.7,0.904861
5,FOREST2,0.0,0.167316,0.0764791,43.592,0.869604
6,ID3,1.66821e-6,0.645887,0.359885,1482.6,0.55595
7,RankTree,2.29379e-6,0.102886,0.050938,59.5,0.916472
8,C45,0.0,0.0859307,0.0460317,65.95,0.928086
9,MON_CART,1.45968e-6,0.122439,0.0593795,76.1,0.900988


### Yeast

In [95]:
resultat3

Row,Methode,INM,MAE,MZE,leaves,tau
,String,Float64,Float64,Float64,Float64,Float64
1,MON_ID3,0.019381,1.32697,0.638427,869.65,-0.0471202
2,MON_C45,0.0203578,1.09955,0.528876,359.1,0.10707
3,CART,0.0214511,1.01247,0.50382,319.7,0.140985
4,MON_RankTree,0.0213514,1.11225,0.525056,363.2,0.108071
5,FOREST2,0.0133217,0.819888,0.449775,49.007,0.186764
6,ID3,0.0200435,1.30236,0.629775,940.3,-0.0301802
7,RankTree,0.0216839,1.08292,0.524831,361.95,0.111673
8,C45,0.0214865,1.02169,0.506404,340.75,0.124877
9,MON_CART,0.0206347,0.993371,0.503258,327.2,0.144596
